<a href="https://colab.research.google.com/github/taiwango188/khpy_python_basics/blob/main/ccxt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## install packages
!pip install ccxt
!pip install mplfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
## import packages
from datetime import datetime
import pandas as pd
import mplfinance as mpf
import ccxt,os

## check available exchanges
print(ccxt.exchanges)

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitteam', 'bitvavo', 'bl3p', 'blockchaincom', 'blofin', 'btcalpha', 'btcbox', 'btcmarkets', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseinternational', 'coinbasepro', 'coincheck', 'coinex', 'coinlist', 'coinmate', 'coinmetro', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'htx', 'huobi', 'huobijp', 'hyperliquid', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'luno', 'lykke', 'mercado', 'mexc', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okx', 'onetrading', 'p2b', 'paymium', 'phemex', 'po

# 1.download klines

In [ ]:
exch = ccxt.bingx() # Loads bingx
exch.load_markets()
print (': %d markets loaded!' % len (exch.markets))
pair ="BTC/USDT"
ohlcv = exch.fetch_ohlcv (pair, '1d')
price = pd.DataFrame(ohlcv, columns =['Date', 'Open','High','Low','Close','Volume'])
price['time'] = pd.to_datetime(price['Date'], unit='ms')

# Set datetime as index
price.set_index('time', inplace=True)
price

: 1148 markets loaded!


,Date,Open,High,Low,Close,Volume
time,,,,,,
2022-11-22 16:00:00,1669132800000,16237.02,16659.48,16054.05,16426.89,7640.19
2022-11-23 16:00:00,1669219200000,16426.80,16798.08,16325.50,16555.80,5889.23
2022-11-24 16:00:00,1669305600000,16557.66,16669.64,16349.52,16512.57,4969.92
2022-11-25 16:00:00,1669392000000,16511.67,16696.15,16459.10,16511.35,5092.80
2022-11-26 16:00:00,1669478400000,16514.58,16596.44,16389.94,16571.75,4006.45
...,...,...,...,...,...,...
2024-03-31 16:00:00,1711900800000,70383.00,71366.89,68188.00,68251.85,4454.53
2024-04-01 16:00:00,1711987200000,68207.90,69903.77,64601.18,64889.80,5786.80
2024-04-02 16:00:00,1712073600000,64890.65,66902.17,64497.13,65839.43,5080.00


# 2.Moving Average & Signal

In [ ]:
price['sma'] = price['Close'].rolling(5).mean()
price['lma'] = price['Close'].rolling(10).mean()
price['signal'] = price.apply(lambda x: 1 if x.sma>x.lma else(-1 if x.sma<x.lma else 0) ,axis = 1)

price['return'] = price.Close.pct_change()

price['comp_return'] = (1+ price['return']).cumprod()
price['strategy_return'] = (1+ price['signal'].shift(1)* price['return']).cumprod()
price.tail()

,Date,Open,High,Low,Close,Volume,sma,lma,signal,return,comp_return,strategy_return
time,,,,,,,,,,,,
2024-03-31 16:00:00,1711900800000,70383.00,71366.89,68188.00,68251.85,4454.53,69816.850,68876.835,1,-0.030277,4.154886,1.227516
2024-04-01 16:00:00,1711987200000,68207.90,69903.77,64601.18,64889.80,5786.80,68539.310,68855.582,-1,-0.049259,3.950218,1.167050
2024-04-02 16:00:00,1712073600000,64890.65,66902.17,64497.13,65839.43,5080.00,67873.044,68872.264,-1,0.014635,4.008028,1.149970
2024-04-03 16:00:00,1712160000000,65838.29,67926.90,65069.52,67643.89,4461.27,67401.560,68658.498,-1,0.027407,4.117876,1.118453
2024-04-04 16:00:00,1712246400000,67648.90,69306.63,66388.03,67166.65,3078.49,66758.324,68349.514,-1,-0.007055,4.088823,1.126344


# 3.Compound Return

In [ ]:
price.tail()

,Date,Open,High,Low,Close,Volume,sma,lma,signal,return,comp_return,strategy_return
time,,,,,,,,,,,,
2024-03-31 16:00:00,1711900800000,70383.00,71366.89,68188.00,68251.85,4454.53,69816.850,68876.835,1,-0.030277,4.154886,1.227516
2024-04-01 16:00:00,1711987200000,68207.90,69903.77,64601.18,64889.80,5786.80,68539.310,68855.582,-1,-0.049259,3.950218,1.167050
2024-04-02 16:00:00,1712073600000,64890.65,66902.17,64497.13,65839.43,5080.00,67873.044,68872.264,-1,0.014635,4.008028,1.149970
2024-04-03 16:00:00,1712160000000,65838.29,67926.90,65069.52,67643.89,4461.27,67401.560,68658.498,-1,0.027407,4.117876,1.118453
2024-04-04 16:00:00,1712246400000,67648.90,69306.63,66388.03,67166.65,3078.49,66758.324,68349.514,-1,-0.007055,4.088823,1.126344
